In [1]:
import numpy as np
import random
from scipy.spatial import distance


#Store Data Variables
import json
with open('feature_data.json', 'r') as f:
 features = json.load(f)

from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
filelist = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
camId = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

In [2]:
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [3]:
query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [4]:
labels

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [5]:
gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [6]:
filelist

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [7]:
camId

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [8]:
#grid searc cv against k, distance metric and transformation/normalization - use only query and gallery for this part 

In [9]:
X = np.array(features)
y = np.array(labels)
filelist = np.array(filelist)
camId = np.array(camId)

In [10]:
mask_train = np.array(train_idxs).ravel()
mask_query = np.array(query_idxs).ravel()
mask_gallery = np.array(gallery_idxs).ravel()

mask_train = np.subtract(mask_train, 1)
mask_query = np.subtract(mask_query, 1)
mask_gallery = np.subtract(mask_gallery, 1)


X_train, X_query, X_gallery = X[mask_train, :], X[mask_query, :], X[mask_gallery, :]
y_train, y_query, y_gallery = y[mask_train], y[mask_query], y[mask_gallery]
filelist_train, filelist_query, filelist_gallery = filelist[mask_train], filelist[mask_query], filelist[mask_gallery]
camId_train, camId_query, camId_gallery = camId[mask_train], camId[mask_query], camId[mask_gallery]


In [11]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(100,), activation='logistic', max_iter=10, alpha=1e-4,
                    solver='adam', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)

In [12]:
for Xtnn, ytnn, camId_t in zip(X_train, y_train, camId_train):
    X_train_pairs = []
    y_train_pair_lables = []
    for Xtnn2, ytnn2, camId_t2 in zip(X_train, y_train, camId_train):
        if ((camId_t == camId_t2) and (ytnn == ytnn2)):
            continue
        elif (ytnn == ytnn2):
            for i in range (30):
                randindex = random.randint(0, 7367)
                Xtnn3 = X_train[randindex]
                ytnn3 = y_train[randindex]                 

                dist2 = 0
                if(ytnn == ytnn3):
                    dist3 = 0
                else:
                    dist3 = distance.euclidean(Xtnn, Xtnn3)

                Xconcat = np.concatenate((Xtnn,Xtnn2,Xtnn3), axis = None)
                X_train_pairs.append(Xconcat)
                y_train_pair_lables.append((dist2,dist3))   

                Xconcat = np.concatenate((Xtnn,Xtnn3,Xtnn2), axis = None)
                X_train_pairs.append(Xconcat)
                y_train_pair_lables.append((dist3,dist2)) 

    y_train_pair_lables = np.array(y_train_pair_lables)
    X_train_pairs = np.array(X_train_pairs)
    if (X_train_pairs.shape[0] != 0):
            mlp.partial_fit(X_train_pairs, y_train_pair_lables)    

Iteration 1, loss = 2349.36911026
Iteration 2, loss = 1790.22884516
Iteration 3, loss = 1445.44597241
Iteration 4, loss = 1249.14812378
Iteration 5, loss = 1175.83751273
Iteration 6, loss = 1168.88941723
Iteration 7, loss = 1187.64441833
Iteration 8, loss = 1174.73610829
Iteration 9, loss = 1135.42468897
Iteration 10, loss = 1073.33191744
Iteration 11, loss = 1015.32964518
Iteration 12, loss = 952.40623705
Iteration 13, loss = 911.56838917
Iteration 14, loss = 883.56032197
Iteration 15, loss = 863.43791907
Iteration 16, loss = 844.81637236
Iteration 17, loss = 815.38542374
Iteration 18, loss = 785.78779244
Iteration 19, loss = 738.86154531
Iteration 20, loss = 704.30615499
Iteration 21, loss = 968.36755473
Iteration 22, loss = 884.42994645
Iteration 23, loss = 774.98273365
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 24, loss = 751.78829676
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


Iteration 116, loss = 302.38143230
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 117, loss = 280.38454579
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 118, loss = 376.58889652
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 119, loss = 325.66887383
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 120, loss = 324.61978588
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 121, loss = 272.32305908
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 122, loss = 240.61813742
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 123, loss = 215.37444129
Training loss did not improve more than tol=0.000100 for two consecutive epochs. St

Iteration 183, loss = 185.27247615
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 184, loss = 110.32149654
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 185, loss = 187.91554702
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 186, loss = 158.14927010
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 187, loss = 189.96423369
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 188, loss = 141.21515141
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 189, loss = 162.40402644
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 190, loss = 171.52980339
Training loss did not improve more than tol=0.000100 for two consecutive epochs. St

Iteration 256, loss = 69.37037794
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 257, loss = 131.28108609
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 258, loss = 133.32600073
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 259, loss = 124.33700715
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 260, loss = 106.38774329
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 261, loss = 164.45495174
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 262, loss = 97.43104785
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 263, loss = 98.07202843
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopp

Iteration 323, loss = 497.96273732
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 324, loss = 537.91528381
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 325, loss = 480.14158411
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 326, loss = 514.55913140
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 327, loss = 497.18560198
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 328, loss = 422.74487437
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 329, loss = 552.74828226
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 330, loss = 466.63727977
Training loss did not improve more than tol=0.000100 for two consecutive epochs. St

...

Iteration 7083, loss = 970.56812527
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7084, loss = 1221.43540708
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7085, loss = 1163.03738224
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7086, loss = 1238.90008311
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7087, loss = 1190.05701985
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7088, loss = 1234.57914660
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7089, loss = 812.20481345
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7090, loss = 814.02255349
Training loss did not improve more than tol=0.000100 for two consecuti

Iteration 7148, loss = 1075.05985972
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7149, loss = 1244.61769064
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7150, loss = 1229.44295051
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7151, loss = 1220.74687247
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7152, loss = 1235.64299608
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7153, loss = 1226.80698664
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7154, loss = 1220.81280215
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7155, loss = 1236.77167348
Training loss did not improve more than tol=0.000100 for two consec

Iteration 7214, loss = 1235.86232293
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7215, loss = 1229.04536415
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7216, loss = 1031.47710631
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7217, loss = 1052.46527337
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7218, loss = 1056.06074462
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7219, loss = 1024.67206298
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7220, loss = 1242.96189911
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7221, loss = 1243.44548616
Training loss did not improve more than tol=0.000100 for two consec

Iteration 7280, loss = 1095.99488888
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7281, loss = 1083.96887895
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7282, loss = 1094.42538529
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7283, loss = 1016.83188982
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7284, loss = 1180.35608289
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7285, loss = 1175.49221658
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7286, loss = 1111.39704868
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7287, loss = 1117.17823750
Training loss did not improve more than tol=0.000100 for two consec

Iteration 7345, loss = 936.10862738
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7346, loss = 1119.11153141
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7347, loss = 1233.35354680
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7348, loss = 1188.85053124
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7349, loss = 1198.23914260
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7350, loss = 1123.38936918
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7351, loss = 1116.85378654
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7352, loss = 1169.58372161
Training loss did not improve more than tol=0.000100 for two consecu

In [38]:
def get_acc_score(y_valid, y_q, tot_label_occur):
    recall = 0
    true_positives = 0
    
    k = 0
    
    max_rank = 30
    
    rank_A = np.zeros(max_rank)
    AP_arr = np.zeros(11)
    
    while (recall < 1) or (k < max_rank):
        if (y_valid[k] == y_q):
            
            true_positives = true_positives + 1
            recall = true_positives/tot_label_occur
            precision = true_positives/(k+1)
            
            AP_arr[round((recall-0.05)*10)] = precision
            
            for n in range (k, max_rank):
                rank_A[n] = 1
            
        k = k+1
        
    max_precision = 0
    for i in range(10, -1, -1):
        max_precision = max(max_precision, AP_arr[i])
        AP_arr[i] = max_precision
    
    AP_ = AP_arr.sum()/11
    
    return AP_, rank_A

In [43]:
from scipy.spatial import distance

def evaluate_metric(X_query, camId_query, y_query, X_gallery, camId_gallery, y_gallery, metric = mlp, parameters = None):

    rank_accuracies = []
    AP = []

    # Break condition for testing
    #q = 0

    for query, camId_q, y_q in zip(X_query, camId_query, y_query):
        q_g_dists = []
        y_valid = []
        for gallery, camId_g, y_g  in zip(X_gallery, camId_gallery, y_gallery):
            if ((camId_q == camId_g) and (y_q == y_g)):
                continue
            else:
                if metric == mlp:
                    dist = mlp.predict(np.concatenate((query, gallery, np.zeros(2048))).reshape((1,6144)))[0][0]
                elif metric == 'minkowski':
                    dist = distance.minkowski(query, gallery, parameters)
                else:
                    raise NameError('Specified metric not supported')           
                q_g_dists.append(dist)
                y_valid.append(y_g)
    
        tot_label_occur = y_valid.count(y_q)
    
        q_g_dists = np.array(q_g_dists)
        print(q_g_dists)
        y_valid = np.array(y_valid)
        
    
        _indexes = np.argsort(q_g_dists)
    
        # Sorted distances and labels
        q_g_dists, y_valid = q_g_dists[_indexes], y_valid[_indexes]
    
        AP_, rank_A = get_acc_score(y_valid, y_q, tot_label_occur)
    
        AP.append(AP_)
        
        rank_accuracies.append(rank_A) 
    
        #if q  > 5:
        #    break
        #q = q+1

    rank_accuracies = np.array(rank_accuracies)

    total = rank_accuracies.shape[0]
    rank_accuracies = rank_accuracies.sum(axis = 0)
    rank_accuracies = np.divide(rank_accuracies, total)

    i = 0
    print ('Accuracies by Rank:')
    while i < rank_accuracies.shape[0]:
        print('Rank ', i+1, ' = %.2f%%' % (rank_accuracies[i] * 100), '\t',
              'Rank ', i+2, ' = %.2f%%' % (rank_accuracies[i+1] * 100), '\t',
              'Rank ', i+3, ' = %.2f%%' % (rank_accuracies[i+2] * 100), '\t',
              'Rank ', i+4, ' = %.2f%%' % (rank_accuracies[i+3] * 100), '\t',
              'Rank ', i+5, ' = %.2f%%' % (rank_accuracies[i+4] * 100))
        i = i+5

    AP = np.array(AP)

    mAP = AP.sum()/AP.shape[0]
    print('mAP = %.2f%%' % (mAP * 100))
    
    return rank_accuracies, mAP

In [44]:
rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric = mlp,
                                       parameters = None)

[60.18071779 60.18071778 60.18071777 ... 71.66304207 60.18071777
 71.66312719]
[71.66312719 71.66312719 71.66312719 ... 71.66312719 60.18071777
 71.66312719]
[60.2256504  71.66312784 71.66312731 ... 60.18071777 60.18071777
 60.18071777]
[71.66312723 71.66312784 71.66312775 ... 60.18071777 60.18071778
 60.18071779]
[60.18071777 60.18071777 60.18071777 ... 60.18071777 60.18071777
 60.18071777]
[60.18071777 60.18071777 60.18071777 ... 60.18071777 60.18071777
 60.18071777]
[60.18071777 60.18071777 60.18071777 ... 60.18071777 60.18071777
 60.18071777]
[60.18071777 60.18071777 60.18071777 ... 60.18071777 60.18071777
 60.18071777]
[71.6631272  71.66312725 71.66312726 ... 60.18071777 60.18071777
 71.66312705]
[60.18071778 60.18071779 60.18071787 ... 60.18071777 60.18071777
 60.18071777]
[60.18082847 60.21917644 60.18238266 ... 60.18071781 60.18071788
 60.18071778]
[60.18071778 60.18072    60.18071787 ... 60.18071777 60.18071777
 60.18071777]
[60.18071777 60.18071779 60.18071777 ... 60.18071777

KeyboardInterrupt: 